In [ ]:
%%time
for batch_ind, data in enumerate(train_loader):
  pass

In [ ]:
# @title Training loop { display-mode: "form" }

import torch.optim as optim

loss_func = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.8)

import torch
import datetime
from torch.utils.tensorboard import SummaryWriter

def calculate_epoch(writer: SummaryWriter, epoch, is_train=False):
  if is_train:
    model.train(True)
  else:
    model.eval()

  total = 0
  correct = 0
  running_loss = 0
  for batch_ind, data in enumerate(train_loader):
    inputs, targets = data
    inputs, targets = inputs.to(device), targets.to(device)

    optimizer.zero_grad()
    output = model(inputs)

    total += len(output)
    correct += (output.argmax() == targets).float().sum()

    targets = torch.nn.functional.one_hot(targets, num_classes = 2).to(torch.float)
    loss = loss_func(output, targets)
    if is_train:
      loss.backward()
      optimizer.step()

    running_loss += loss.item()

    if batch_ind % 10 == 9:
      stage = "train" if is_train else "test"
      avg_loss = running_loss / 10
      writer.add_scalar(f"loss/{stage}", avg_loss, epoch*batch_size + batch_ind)
      writer.add_scalar(f"accuracy/{stage}", correct/total, epoch*batch_size + batch_ind)
      running_loss = 0


def train_loop():
  time_now = datetime.datetime.now()
  layout = {
      "Model statistics" : {
      "loss": ["Multiline", ["loss/train", "loss/test"]],
      "accuracy": ["Multiline", ["accuracy/train", "accuracy/test"]]
      }
  }
  writer = SummaryWriter(log_dir=time_now.strftime("C:\\Users\\bes-s\\OneDrive\\Документы\\NN\\runs\\run %d-%m %H_%M_%S"))
  writer.add_custom_scalars(layout)
  epoch_count = 100
  for epoch in range(epoch_count):  # loop over the dataset multiple times
    running_loss = 0.0
    print(f"epoch #{epoch+1}/{epoch_count}...", end = "")
    calculate_epoch(writer, epoch, is_train=True)
    with torch.no_grad():
      calculate_epoch(writer, epoch, is_train=False)
    print("done.")
    writer.flush()

  writer.close()
  print('done training.')


train_loop()